In [1]:
%reset -f

In [2]:
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn import svm, datasets, feature_selection, cross_validation
from sklearn.feature_selection import SelectKBest, f_regression, SelectFdr, SelectFpr
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, RFECV, RFE, mutual_info_classif
from sklearn.feature_selection import VarianceThreshold, f_regression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import validation_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib
from sklearn.metrics import f1_score, precision_score
from sklearn.svm import LinearSVC
from sklearn import feature_selection
from sklearn import cross_validation
from sklearn.externals import joblib
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,OneHotEncoder,Imputer,Normalizer
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.externals import joblib
from sklearn.decomposition import PCA, RandomizedPCA, KernelPCA
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.learning_curve import learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.externals import joblib
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LassoCV
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from scipy.stats import mannwhitneyu

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.classifier import StackingClassifier
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from pylab import pcolor, show, colorbar, xticks, yticks
from numpy import corrcoef, sum, log, arange
from scipy.stats import pearsonr, spearmanr
from multiprocessing import Pool
import matplotlib.pyplot as plt
import logging.config
import seaborn as sns
import pandas as pd
import numpy as np 
import datetime
import time
import csv
import os
import pickle
%matplotlib inline

import sys
sys.path.insert(1, "/home/aegle/projects/myosa/src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score
from UnivCombineFilter import UnivCombineFilter

/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. Th

In [3]:
ds = 2
suffix = ''

In [4]:
datafilenames = []
datafilenames.append(os.path.join('resources','data_partition_cl0_short_no_monit_14012016{}.pkl'.format(suffix)))
datafilenames.append(os.path.join('resources','data_partition_cl0_short_1m_monit_14012016{}.pkl'.format(suffix)))
datafilenames.append(os.path.join('resources','data_partition_cl0_short_3m_1m_monit_14012016{}.pkl'.format(suffix)))


#Load Train & test data
f = file(datafilenames[ds],"rb")
X_train = np.load(f).astype(float)
y_train = np.load(f).astype(int)
X_test = np.load(f).astype(float)
y_test = np.load(f).astype(int)
cols = np.load(f)
tr_codes = np.load(f)
ts_codes = np.load(f)
feat_types = np.load(f)
feat_types = dict(feat_types.tolist())
f.close()

#Join data
X_all = np.vstack((X_train,X_test))
y_all = np.hstack((y_train, y_test)).reshape(-1,1)
all_codes = np.hstack((tr_codes,ts_codes))
print X_all.shape, y_all.shape, all_codes.shape
print X_train.shape, X_test.shape
print cols

(42, 70) (42, 1) (42,)
(29, 70) (13, 70)
['GENERO' 'ACTIVO' 'JUBILADO' 'EDAD' 'FUMA' 'FUMA_PAQ_ANY' 'FUMA_EX_ANYS'
 'ALCOHOL' 'ALCOHOL_GR_DIA' 'CAFES_DIA' 'DEPRESION' 'ANSIEDAD' 'HTA'
 'CARDIOPATIA' 'ENF_RESP' 'DIABETES' 'OBESIDAD' 'DISLIPEMIA'
 'OBSTR_NASAL_CRON' 'RONCA' 'SUE_REP' 'CRISIS_ASF' 'NICTURIA' 'APNEAS_PRES'
 'CEFALEA' 'SOMN_DIURN' 'DESPERTAR_NOCT' 'TRAST_CONC' 'IRR_APAT_DEPR'
 'INSOMNIO' 'ACT_MOTR_NOCT' 'SEN_SUE_REAL_DORM' 'SUE_INT_PIROSIS'
 'DISM_DESEO_SEX' 'IECAS' 'DIURETICOS' 'ANTIAGREG' 'ANTIACID' 'HIPOLIPEM'
 'BDZ' 'TIEMP_SUENO' 'IAH' 'TIEMPOSAT<90%' 'IND_DESAT' 'TALLA' 'IMC'
 'CIRC_CUELLO' 'MEDIA_TAS' 'MEDIA_TAD' 'Sat O2' 'PRESS_CPAP' 'EPWORTH'
 'EUROQOL' 'VISUAL' 'HORAS_USO_TOT_1' 'HORAS_USO_MED_NOCHE_1'
 'AL_IRR_CUT_MASC_1' 'BOCA_SECA_1' 'MEDIA_TAS_1' 'MEDIA_TAD_1' 'EPWORTH_1'
 'EUROQOL_1' 'VISUAL_EUROQOL_1' 'HORAS_USO_MED_NOCHE_3' 'BOCA_SECA_3'
 'MEDIA TAS_3' 'MEDIA TAD_3' 'EPWORTH_3' 'EUROQOL_3' 'VISUAL EUROQOL_3'
 'LABEL']


#### Find categorical (discrete) columns

In [5]:
catCols = []
reducedCols = cols[:-1]

for i in range(len(cols)-1):
    if feat_types[cols[i]] == 'object' \
        or cols[i] in ("RONCA","SUE_REP","CRISIS_ASF","NICTURIA","APNEAS_PRES","CEFALEA",
                "SOMN_DIURN","DESPERTAR_NOCT","TRAST_MEM","TRAST_CONC","IRR_APAT_DEPR"):
        catCols.append(1)
    else:
        catCols.append(0)
catCols = np.array(catCols)

print "Cat cols:", np.sum(catCols==1)
print "Num cols:", np.sum(catCols==0)
print len(reducedCols)

Cat cols: 38
Num cols: 32
70


### 1. Clean irrelevant features (Optional)

In [6]:
to_del_name = []
to_del_ix = []

#### Null values > 0.3

In [7]:
#Find columns with NaN   
naUsers = np.hstack((all_codes.reshape((-1,1)), \
          (np.sum(np.isnan(X_all.tolist()), axis=1) / float(X_all.shape[1])).reshape(len(X_all),1)))

df_naUsers = pd.DataFrame(data = naUsers, columns = ['users', 'NumNaNCols'])
print "Number of NaN by individual > 30%:", np.sum(df_naUsers["NumNaNCols"] > 0.3)

#Filter columns with NAN > 30%    
nanColumns = np.sum(np.isnan(X_all.tolist()), axis=0) / float(X_all.shape[0])
naVals = np.hstack((cols[:-1].reshape(-1,1), nanColumns.reshape(-1,1)))
df_naVals = pd.DataFrame(data = naVals, columns = ['columnNames', 'NaN_perc'])
print "Number of NaN by column > 30%:", np.sum(df_naVals["NaN_perc"] > 0.3)

to_del = []
for h, col in enumerate(nanColumns):
    if col > 0.3:
        print "DELETE column with high NAN values (30%):", cols[h]
        to_del.append(cols[h])

if len(to_del) == 0:
    print "No features to del with high NAN values"

Number of NaN by individual > 30%: 0
Number of NaN by column > 30%: 0
No features to del with high NAN values


#### Imput nan values

In [8]:
imputer = TypeFeatImputer(catCols, reducedCols)
imputer.fit(X_all, y_all)
X_Imp = imputer.transform(X_all)
X_filt = X_Imp

print "Nulls after imputation:", np.sum(np.sum(np.isnan(X_Imp)))

Nulls after imputation: 0


#### Low variant features Filter

In [9]:
varFilter = VarianceThreshold(threshold=0.0)
varFilter.fit(X_Imp)

to_del_name.extend(cols[:-1][varFilter.get_support() == 0])
to_del_ix.extend(np.where(varFilter.get_support() == 0)[0])

print X_filt.shape
print "Filtered columns:", np.sum(varFilter.get_support()==False)
print to_del_ix

(42, 70)
Filtered columns: 0
[]


#### Highly correlated features

In [10]:
cols_corr = []
p_scores = []
p_scores_all = []
v_th = 0.15

for i in range(X_Imp.shape[1]):
    for j in range(X_Imp.shape[1]):
        if i != j and (i,j) not in cols_corr and (j,i) not in cols_corr:
            
            cols_corr.append((i,j))
            coeff_corr, p_score = spearmanr(X_Imp[:,i], X_Imp[:,j])
            
            if float(coeff_corr) > (1.0 - v_th) or float(coeff_corr) < (v_th - 1.0) :
                sc1, pval1 = mannwhitneyu(X_Imp[:,i].reshape(-1,1),y_all.reshape(-1,1))
                sc2, pval2 = mannwhitneyu(X_Imp[:,j].reshape(-1,1),y_all.reshape(-1,1))
                
                print "\nSimilar features",cols[i],"and",cols[j],"score:", coeff_corr
                print "Class significance of", cols[i], "is", pval1
                print "Class significance of", cols[j], "is", pval2
                
                to_del_ix.append(j if pval1 < pval2 else i)
                to_del_name.append(cols[j if pval1 < pval2 else i])
                p_scores.append([i, j, coeff_corr, p_score])
                

print "Columns to delete:", len(to_del_name), to_del_name
print to_del_ix


Similar features ACTIVO and JUBILADO score: -0.857492925713
Class significance of ACTIVO is 0.414979503738
Class significance of JUBILADO is 0.0148575446864

Similar features ALCOHOL and ALCOHOL_GR_DIA score: 0.874313210259
Class significance of ALCOHOL is 0.0974316203361
Class significance of ALCOHOL_GR_DIA is 0.27947537567
Columns to delete: 2 ['ACTIVO', 'ALCOHOL_GR_DIA']
[1, 8]


#### Delete columns

In [30]:
if len(to_del_name) > 0:

    clean_cols_ix = [i for i in range(X_train.shape[1]) if i not in to_del_ix]
    clean_cols_name = cols[clean_cols_ix]
    
    print "Clean cols:", len(clean_cols_ix)

    catCols = []
    for i in range(len(clean_cols_name)):
        if feat_types[clean_cols_name[i]] == 'object' \
            or cols[i] in ("RONCA","SUE_REP","CRISIS_ASF","NICTURIA","APNEAS_PRES","CEFALEA",
                    "SOMN_DIURN","DESPERTAR_NOCT","TRAST_MEM","TRAST_CONC","IRR_APAT_DEPR"):
            catCols.append(1)
        else:
            catCols.append(0)
            
    catCols = np.array(catCols)
    cols = clean_cols_name.tolist()
    cols.append(["LABEL"])    
    reducedCols = cols[:-1]
    
    X_train = X_train[:, clean_cols_ix]
    X_test = X_test[:, clean_cols_ix]
    X_filt = X_filt[:,clean_cols_ix]
    
    print "Cat cols:", np.sum(catCols==1)
    print "Num cols:", np.sum(catCols==0)
    print len(reducedCols)
    print X_train.shape
    print X_test.shape
    print X_filt.shape
    print cols

### 2. Outlier filtering (Optional)

In [12]:
outFilter = OutlierFiltering(cols,all_codes,norm="l2")
outFilter.fit(X_filt.astype(float), y_all)
toDel =  np.hstack((outFilter.codesToDel))

0.187628211208 9.74966275351 9.93729096472
Ouliers min ratio: 10.2187332815 9.4682204367
Num outliers detected: 1
Num outliers detected: 2
[('MY02', 10.056445015986377), ('MY26', 9.9323251930368119), ('MY28', 9.8226505975830047), ('MY37', 10.009306444770054), ('MY33', 9.8559365243865784), ('MY24', 10.06015584937227), ('MY12', 9.5955126410019957), ('MY50', 9.9242615267190466), ('MY42', 9.6932723135795058), ('MY45', 9.670967204896959), ('MY29', 10.092544597354381), ('MY32', 9.7452672219234486), ('MY57', 9.8496239597285626), ('MY56', 9.8678628844699787), ('MY46', 9.8472200465759485), ('MY53', 9.9389462219459528), ('MY54', 9.602810325960931), ('MY27', 9.8423222925064415), ('MY19', 10.097004589946019), ('MY21', 9.8582194532184513), ('MY61', 9.837804354081781), ('MY14', 9.8349152659842716), ('MY55', 10.53251673271707), ('MY06', 9.7848204828974854), ('MY51', 9.7628493482716863), ('MY05', 9.9750763193102987), ('MY18', 9.6988707037426956), ('MY59', 9.7958130009393347), ('MY01', 9.85623596881203

In [13]:
#Remove outliers from train and add them to test

ix_tr = [i for i in range(len(tr_codes)) for d in toDel if tr_codes[i] == d]
print ix_tr
print tr_codes[ix_tr]

ix_ts = [i for i in range(len(ts_codes)) for d in toDel if ts_codes[i] == d]
print ix_ts
print ts_codes[ix_ts]

ix_all = np.hstack((ix_tr,ix_ts))
print ix_all

#Change anomal row from train to test (if not anomaly and same label type (to assure stratification))
ix_news = []
for i in range(len(ix_tr)):
    for j in range(len(ts_codes)):
        if (j not in ix_ts) and (j not in ix_news) and y_train[ix_tr[i]] == y_test[j]:
            ix_news.append(j)
            break
        
print "ix_new:", ix_news
print len(tr_codes)


#Remove outliers individuals from train
X_test = np.vstack((X_test, X_train[ix_tr,:]))
y_test = np.hstack((y_test, y_train[ix_tr]))
ts_codes = np.hstack((ts_codes,tr_codes[ix_tr]))

X_train = np.delete(X_train, (ix_tr), axis=0)
y_train  = np.delete(y_train, (ix_tr), axis=0)
tr_codes = np.delete(tr_codes,(ix_tr), axis=0)
    
trick = True
if trick:    
    #Add healthy individual from test to train  
    for ix_new in ix_news:        
        X_train = np.vstack((X_train, X_test[ix_new,:]))
        y_train = np.hstack((y_train, y_test[ix_new]))
        tr_codes = np.hstack((tr_codes,ts_codes[ix_new]))

        X_test = np.delete(X_test, (ix_new), axis=0)
        y_test  = np.delete(y_test, (ix_new), axis=0)
        ts_codes = np.delete(ts_codes,(ix_new), axis=0)
    

print "Train:", X_train.shape
print "Test:", X_test.shape
print tr_codes
print ts_codes

[22]
['MY55']
[7, 10]
['MY09' 'MY35']
[22  7 10]
ix_new: [1]
29
Train: (29, 68)
Test: (13, 68)
['MY02' 'MY26' 'MY28' 'MY37' 'MY33' 'MY24' 'MY12' 'MY50' 'MY42' 'MY45'
 'MY29' 'MY32' 'MY57' 'MY56' 'MY46' 'MY53' 'MY54' 'MY27' 'MY19' 'MY21'
 'MY61' 'MY14' 'MY06' 'MY51' 'MY05' 'MY18' 'MY59' 'MY01' 'MY20']
['MY22' 'MY47' 'MY17' 'MY41' 'MY03' 'MY34' 'MY09' 'MY36' 'MY44' 'MY35'
 'MY15' 'MY48' 'MY55']


#### PRE-Select features (Optional)

In [29]:
#dfFeat = joblib.load(os.path.join('resources','rfecv_cl0_short_3m_1m_monit_14012016.pkl'))
#print np.sum(dfFeat.get_support() == 1)

#selCols = ['JUBILADO', 'FUMA_PAQ_ANY', 'HTA' ,'CARDIOPATIA', 'DIABETES', 'DISLIPEMIA',
# 'DESPERTAR_NOCT', 'INSOMNIO', 'DOLOR_PIER_MOVER', 'SEN_SUE_REAL_DORM',
# 'DISM_DESEO_SEX', 'BBLOQ', 'INSULINA', 'BDZ', 'OTROS_HIPOT', 'IMC', 'CIRC_ABD',
# 'AL_IRR_CUT_MAS_3', 'BOCA_SECA_3', 'EPWORTH_3', 'VISUAL EUROQOL_3']
#selCols = ['BOCA_SECA_3', 'INSOMNIO', 'DOLOR_PIER_MOVER', 'FUMA', 'EUROQOL_1', 'VISUAL']

selCols = ['INSOMNIO', 'CIRC_ABD', 'BBLOQ', 'ANSIEDAD', 'CARDIOPATIA', 'DIABETES', 'DISLIPEMIA', 
           'OTROS_PSICO', 'TIEMP_SUENO', 'PESO','DOLOR_PIER_MOVER', 'MOV_BRUSC_INV_DUERME', 'MEDIA_TAD', 
           'FUMA_EX_ANYS', 'ALCOHOL_GR_DIA', 'ANTIAGREG', 'OTROS_HIPOT', 'ENF_NEUR', 'CEFALEA', 'FUMA']

print len(selCols)

filtSelCols = [c for i,c in enumerate(cols) if c in selCols] 
print len(filtSelCols)

if len(selCols) > 0:

    clean_cols_ix = [i for i,c in enumerate(cols) if c in filtSelCols]    
    clean_cols_name = filtSelCols
    
    print "Clean cols:", len(clean_cols_ix)

    catCols = []
    for i in range(len(clean_cols_name)):
        if feat_types[clean_cols_name[i]] == 'object' \
            or clean_cols_name[i] in ("RONCA","SUE_REP","CRISIS_ASF","NICTURIA","APNEAS_PRES","CEFALEA",
                    "SOMN_DIURN","DESPERTAR_NOCT","TRAST_MEM","TRAST_CONC","IRR_APAT_DEPR"):
            catCols.append(1)
        else:
            catCols.append(0)
            
    catCols = np.array(catCols)
    cols = clean_cols_name
    cols.append(["LABEL"])
    reducedCols = cols[:-1]
    
    X_train = X_train[:, clean_cols_ix]
    X_test = X_test[:, clean_cols_ix]
    X_filt = X_filt[:,clean_cols_ix]
    
    print "Cat cols:", np.sum(catCols==1)
    print "Num cols:", np.sum(catCols==0)
    print len(catCols)
    print len(cols)
    print len(reducedCols)
    print X_train.shape
    print X_test.shape
    print X_filt.shape
    

20
11
Clean cols: 11
Cat cols: 8
Num cols: 3
11
12
11
(29, 11)
(13, 11)
(42, 11)


#### Config pipelines

In [55]:
fs_methods = ["none"] 
#"rfe_logReg_fs","rfe_rf_fs","mi_fs","iscore_fs", "discrete_fs", "continuous_fs",
#"none","combine_fs","fusion_fs",sfs_fs,lasso_fs
cls_method = "nn"
sm_method = "none" # sm_smote, none
sm_type = "after"# before, after

#### Create pipelines

In [56]:
all_ds = [0,1,2]
suffixes = ['_no_out','']
metrics = ["precision_weighted","f1_weighted"]
all_fs_methods = ["no_fs", "extraTrees_fs", "rfe_fs",
                  "lasso_fs", "ftest_fs","fpr_fs", "fdr_fs",
                  "mi_fs","randLasso_fs"]
samplers_types = ["none", "before", "after"]
samplers_methods = ["none","sm_smote", "sm_ada"]
all_cls_methods = ["svmRBF","logReg","knn","rf","nb","vot","nn"]

In [57]:
basePipeline = Pipeline([
        ("Imputer", TypeFeatImputer(catCols, reducedCols)),
        ("Scaler", StandardScaler()),
        ("Variance", VarianceThreshold(threshold=0.0))
    ])

params = {}
pipeline = []
pipe = Pipeline(list(basePipeline.steps))

#Feat selection
for fs_method in fs_methods:
    
    if fs_method == "sfs_fs":
        knn = KNeighborsClassifier(n_neighbors=3)
        rf = RandomForestClassifier(class_weight=None,n_estimators=50,criterion='gini',random_state=33)
        sfs = SFS(knn, 
                  k_features=6, 
                  forward=True, 
                  floating=True, 
                  scoring='f1_weighted',
                  verbose=0,
                  n_jobs = -1,
                  cv=3)
        pipe.steps.append((fs_method,sfs))

    if fs_method == "combine_mix_fs":
        pipe.steps.append((fs_method, \
            FeatureUnion(transformer_list=[
            ('disc_pipe', Pipeline([\
            ("disc", DiscreteFilter(catCols)),\
            #("disc_sfm", SelectFromModel(RandomForestClassifier(n_estimators=50,random_state=33)))\
            ("disc_sfm", SelectKBest(feature_selection.chi2))\
            ])),\
            ('cont_pipe', Pipeline([\
            ("cont", ContinuousFilter(catCols)),\
            #("cont_sfm", SelectFromModel(LogisticRegression(penalty="l1", dual=False, random_state=42)))\
            ("cont_sfm", SelectKBest(feature_selection.mutual_info_classif))\
            ]))\
            ])\
            ))
        params.update({fs_method + "__disc_pipe__disc_sfm__k":[5,7,10]})  
        params.update({fs_method + "__cont_pipe__cont_sfm__k":[2,5,7]})     
        
    if fs_method == "combine_fs":
        pipe.steps.insert(1,(fs_method, UnivCombineFilter(catCols,np.array(reducedCols))))
        params.update({fs_method + '__percentile':[10,15,20,25,30,50]})
        
    if fs_method == "combine_old_fs":
        pipe.steps.insert(1,(fs_method, FeatureUnion([("disc", DiscreteFS(catCols,oneMinFeat=False)), 
                                                    ("cont", ContinuousFS(catCols,oneMinFeat=False))])))
    if fs_method == "discrete_fs":
        pipe.steps.append((fs_method, DiscreteFS(catCols)))
        
    if fs_method == "continuous_fs":
        pipe.steps.append((fs_method, ContinuousFS(catCols)))
        
    if fs_method == "rfe_rf_fs":
        pipe.steps.append((fs_method, RFE(estimator=RandomForestClassifier(class_weight=None,
                                                                           n_estimators=50,
                                                                           criterion='gini',
                                                                           random_state=33))))
        params.update({fs_method + '__step':[0.1]})
        params.update({fs_method + '__n_features_to_select': #[10,13]}) #7,9,10,11,
                                        [int(len(reducedCols)*0.2), 
                                        int(len(reducedCols)*0.4), 
                                        int(len(reducedCols)*0.6), 
                                        int(len(reducedCols)*0.8)]})        

    if fs_method == "rfe_logReg_fs":

        pipe.steps.append((fs_method, RFE(estimator=LogisticRegression(penalty="l1",random_state=33))))
        #params.update({fs_method + '__step':[1]})
        #params.update({fs_method + '__n_features_to_select': [7,9,10,11,13]})
        params.update({fs_method + '__step':[0.1]})
        params.update({fs_method + '__n_features_to_select':[int(len(reducedCols)*0.2), 
                                        int(len(reducedCols)*0.4), 
                                        int(len(reducedCols)*0.6), 
                                        int(len(reducedCols)*0.8)]})
        #params.update({fs_method + '__estimator__C':[0.001,0.001,0.01,0.1,1]})
        #params.update({fs_method + '__estimator__class_weight':[None,'balanced']})
                          
                           
    if fs_method == "ftest_fs":
        pipe.steps.append((fs_method, SelectKBest(feature_selection.f_classif)))
        params.update({fs_method + '__k':[7,10,15,20]})

    if fs_method == "mi_fs":
        pipe.steps.append((fs_method, SelectKBest(feature_selection.mutual_info_classif)))
        params.update({fs_method + '__k':[7,10,15,20]})

    if fs_method == "iscore_fs":
        pipe.steps.append((fs_method, SelectKBest(i_score)))
        params.update({fs_method + '__k':[7,10,15,20]})    
        
    if fs_method == "fdr_fs":
        pipe.steps.append((fs_method, SelectFdr(f_classif)))

    if fs_method == "fpr_fs":
        pipe.steps.append((fs_method, SelectFpr(f_classif)))

    if fs_method == 'extraTrees_fs':
        pipe.steps.append((fs_method, SelectFromModel(
                    RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=24))))

    if fs_method == 'lasso_fs':
        pipe.steps.append((fs_method, SelectFromModel(
                    LogisticRegression(n_jobs=-1, penalty="l1", dual=False, random_state=42))))
        params.update({fs_method + '__estimator__C': [0.5,1,3]})

    if fs_method == 'randLasso_fs':
        pipe.steps.append((fs_method, 
                    RandomizedLogisticRegression(sample_fraction=1, fit_intercept =False, 
                                                 n_jobs=-1, random_state=42)))



#Add classifiers
if cls_method == "knn":
    pipe.steps.append((cls_method, KNeighborsClassifier()))
    params.update({'knn__n_neighbors':[1,3,5,7,9,11], 'knn__weights':['uniform', 'distance']})

if cls_method == "nb":                              
    pipe.steps.append((cls_method, GaussianNB()))

if cls_method == "logReg":
    pipe.steps.append((cls_method, LogisticRegression(random_state=42)))
    params.update({'logReg__C': [0.00001,0.0001,0.001,0.01,0.1,0.5,1,5,10,50,100]})
    params.update({'logReg__class_weight': [None, 'balanced']})
    params.update({'logReg__penalty': ['l1', 'l2']})
    
if cls_method == "svmLin":
    pipe.steps.append((cls_method, SVC(kernel='linear', probability=True,random_state=42)))
    params.update({cls_method + '__C': [0.00001,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,15,30,50,100]})
    params.update({cls_method + '__class_weight': [None, 'balanced']})
     
if cls_method == "svmRBF":
    pipe.steps.append((cls_method, SVC(random_state=42,probability=True)))
    params.update({'svmRBF__C': [0.001,0.01,0.1,0.5,1,5,10,30,100,150], 
     'svmRBF__gamma' : [0.00001,0.0001,0.001,0.01, 0.1,1]})
    params.update({'svmRBF__class_weight': [None, 'balanced']})

if cls_method == "rf":
    pipe.steps.append((cls_method, RandomForestClassifier(n_jobs=-1,random_state=42)))
    params.update({'rf__n_estimators': [100,200,500,750,1000], 'rf__criterion': ['entropy','gini'],
                 'rf__max_depth' : [None,2,4,5]})
    params.update({'rf__class_weight': [None, 'balanced']})

if cls_method == "nn":
    pipe.steps.append((cls_method, MLPClassifier(
                activation='logistic',
                solver='lbfgs', 
                hidden_layer_sizes=(5, 2), 
                random_state=13)))
    params.update({
            'nn__alpha': [1e-5,0.00001,0.0001,0.001,0.01,0.1,1,3,5,10],
            'nn__hidden_layer_sizes':[(30,),(50,),(70,),(150,),
                                      (30,30),(50,50),(70,70),
                                      (30,30,30),(50,50,50),(70,70,70)
                                     ]
                  })

if cls_method == "ada":
    pipe.steps.append((cls_method, AdaBoostClassifier(random_state=42)))
    params.update({'ada__n_estimators': [5,10,50,100,200],
                 'ada__learning_rate' : [0.01,0.1,0.5,1.0],
                 'ada__base_estimator': [KNeighborsClassifier(n_neighbors=3)]})


    
if cls_method == "vot":
    pipe.steps.append((cls_method, VotingClassifier(voting='soft', estimators=
            [
                ('lr', LogisticRegression(random_state=42,penalty="l1")),
                ('knn', KNeighborsClassifier(n_neighbors=3)),
                ('svm', SVC(kernel="rbf",random_state=42,probability=True)),
                ('rf', RandomForestClassifier(n_estimators=10, n_jobs=-1,random_state=42)),
                #('ada', AdaBoostClassifier(random_state=42)),
                ('nb', GaussianNB()),
            ])))
    
if cls_method == "ensemble":
    clf1 = KNeighborsClassifier(n_neighbors=9)
    clf2 = RandomForestClassifier(n_estimators=200, criterion="gini",class_weight=None,
                                  max_depth=4,random_state=1)
    clf3 = SVC(C=10,gamma=0.001,class_weight=None, probability=True, kernel="rbf",random_state=1)
    clf4 = LogisticRegression(class_weight=None,C=0.5,penalty="l1",random_state=1)
    clf5 = DecisionTreeClassifier(random_state=1)
    
    meta = RandomForestClassifier(n_estimators=100)
    
    sclf = StackingClassifier(classifiers=[cl1,clf2, clf3, clf4,cl5],
                              meta_classifier=meta, use_probas=True, average_probas=True)
    pipe.steps.append((cls_method, sclf))
    
"""
    params.update({cls_method + '__SVC__C': [0.01,0.1,0.5,1,3,5,7,10,30,100], 
                   cls_method + '__gamma' : [0.0001,0.001,0.01, 0.1,1,3],
                   cls_method + '__class_weight': [None, 'balanced']})
    
"""

    

"\n    params.update({cls_method + '__SVC__C': [0.01,0.1,0.5,1,3,5,7,10,30,100], \n                   cls_method + '__gamma' : [0.0001,0.001,0.01, 0.1,1,3],\n                   cls_method + '__class_weight': [None, 'balanced']})\n    \n"

In [58]:
samplers = [
    ("none", None),
    ("sm_smote", SMOTE(ratio='auto', random_state=32)),
    ("sm_ada", ADASYN(random_state=32))]

pipeline = []

#Create pipeline object (avoiding certain tests)
#if not (cls_method == "rf" and fs_method == 'extraTrees_fs'):    
        
for i, sampler_method in enumerate(samplers_methods):
    for sampler_type in samplers_types:            

        #Cast pipeline to imb_pipeline
        pipe_imb = make_pipeline(*[p[1] for p in pipe.steps])
        stps = len(pipe_imb.steps)        
        for s in range(stps):
            pipe_imb.steps.remove(pipe_imb.steps[0])
        for s in range(stps):
            pipe_imb.steps.append(pipe.steps[s])

        #Append samplings
        if str(sampler_method).lower() == sm_method.lower() and sampler_type.lower() == sm_type.lower() \
                and sm_method.lower() == "none":
            pipeline.append([[fs_method for fs_method in fs_methods], cls_method, sm_method, pipe_imb, params])            

        if sampler_method == sm_method and sampler_type == sm_type and sampler_type == "after" \
            and sm_method.lower() != "none":
                
            pipe_imb.steps.insert(stps - 1, samplers[i])
            pipeline.append([[fs_method for fs_method in fs_methods], cls_method, sm_method + "_" + sm_type, pipe_imb, params])
            if sm_method == "sm_smote":
                params.update({sm_method + "__k_neighbors":[3,4,5],
                               sm_method + "__kind":['regular','svm']})
            else:
                params.update({sm_method + "__n_neighbors":[3,4,5],
                               sm_method + "__kind":['regular','svm']})

        if sampler_method == sm_method and sampler_type == sm_type and sampler_type == "before" \
            and sm_method.lower() != "none":
            
            pipe_imb.steps.insert(3, samplers[i])                
            pipeline.append([[fs_method for fs_method in fs_methods], cls_method, sm_method + "_" + sm_type, pipe_imb, params])
            if sm_method == "sm_smote":
                params.update({sm_method + "__k_neighbors":[3,4,5],
                               sm_method + "__kind":['regular','svm']})
            else:
                params.update({sm_method + "__n_neighbors":[3,4,5],
                               sm_method + "__kind":['regular','svm']})
            
            
pipeline = pd.DataFrame(pipeline, columns=["fs","cls","smp","pipe","pipe_params"])
pipeline.sort_values("fs", inplace=True)
pipeline.head()

,fs,cls,smp,pipe,pipe_params
0,[none],nn,none,"Pipeline(steps=[('Imputer', TypeFeatImputer(al...","{u'nn__hidden_layer_sizes': [(30,), (50,), (70..."


#### Config experiment pipeline

In [59]:
num_exp = 0
verbose = False
metric = "f1_weighted"
cv_thr = 0.3

print pipeline.iloc[0,3].steps

[('Imputer', TypeFeatImputer(allNameCols=array(['GENERO', 'ACTIVO', ..., 'EUROQOL_3', 'VISUAL EUROQOL_3'], dtype=object),
        dataCatCols=array([1, 1, ..., 0, 0]))), ('Scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('Variance', VarianceThreshold(threshold=0.0)), ('nn', MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=13, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))]


#### Run single experiment

In [66]:
print "ALL TRAIN:", X_train.shape
print "TRAIN:", "[0's:", np.sum(y_train==0), "1's:", np.sum(y_train==1), "]"
print "ALL TEST:", X_test.shape
print "TEST:", "[0's:", np.sum(y_test==0), "1's:", np.sum(y_test==1), "]"

try:
    
    # Run experiment
    start = time.time()

    #Prepare pipe_cls      
    pipeline_cls = pipeline["pipe"].iloc[num_exp]
    pipeline_params = pipeline["pipe_params"].iloc[num_exp]

    if verbose:
        print "\n",pipeline_cls.steps

    
    #Prepare cv
    cv_inner = StratifiedShuffleSplit(y_train, n_iter=10, test_size=cv_thr,random_state=24)
    cv_outer = StratifiedShuffleSplit(y_train, n_iter=10, test_size=cv_thr,random_state=42)

    #for cv in cv_inner:
    #    print np.sum(y_train[cv[0]] == 0), np.sum(y_train[cv[0]] == 1)
    #    print np.sum(y_train[cv[1]] == 0), np.sum(y_train[cv[1]] == 1)
    
    #Fit pipeline with CV                        
    grid_pipeline = GridSearchCV(pipeline_cls, param_grid=pipeline_params, verbose=1, 
                                 n_jobs=-1, cv=cv_inner, scoring= metric, error_score = 0,
                                 refit=True) 
    grid_pipeline.fit(X_train, y_train)

    # Compute pipeline evaluation with CV
    print "\nCV INNER selected params {}".format(grid_pipeline.best_params_.values())
    
    cv_inner_f1 = cross_validation.cross_val_score(grid_pipeline.best_estimator_, X_train, y_train, 
                                             cv=cv_inner, scoring='f1_weighted', n_jobs=-1)
    
    print "CV {} score: {}".format(metric, grid_pipeline.best_score_)
    print "CV f1 score: %0.3f  (+/-%0.03f)" % (np.mean(cv_inner_f1), np.std(cv_inner_f1))
    
    cv_f1 = cross_validation.cross_val_score(grid_pipeline.best_estimator_, X_train, y_train, 
                                             cv=cv_outer, scoring='f1_weighted', n_jobs=-1)

    print "\nCV OUTER selected params {}".format(grid_pipeline.best_params_.values())
    print "CV f1 score: %0.3f  (+/-%0.03f)" % (np.mean(cv_f1), np.std(cv_f1))
    print "Selected params (bests from CV) {}".format(grid_pipeline.best_params_.values())

    # Computel Train score (with best CV params)
    y_pred = grid_pipeline.best_estimator_.predict(X_train)                
    train_scores = precision_score(y_train, y_pred, average='weighted', pos_label=None)
    train_f1_scores = f1_score(y_train, y_pred, average='weighted', pos_label=None)
    
    print "\nTR Prec score:", train_scores
    print "TR F1 score:", train_f1_scores

    #Compute test score
    y_pred = grid_pipeline.best_estimator_.predict(X_test)
    test_f1 = f1_score(y_test, y_pred, average='weighted', pos_label=None)
    print "\nTest f1: %0.3f" % (test_f1)
    print "with following performance in test:"
    print classification_report(y_test, y_pred)
    print confusion_matrix(y_test, y_pred)

    end = time.time()
    print "Total time:", end - start

except Exception, err:
    print "Error on the experiment", err    

ALL TRAIN: (29, 70)
TRAIN: [0's: 12 1's: 17 ]
ALL TEST: (13, 70)
TEST: [0's: 6 1's: 7 ]
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
8 12
4 5
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    6.5s
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/aegle/miniconda2/envs/myosa/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMet


CV INNER selected params [(70, 70), 0.01]
CV f1_weighted score: 0.759090909091
CV f1 score: 0.759  (+/-0.151)

CV OUTER selected params [(70, 70), 0.01]
CV f1 score: 0.827  (+/-0.081)
Selected params (bests from CV) [(70, 70), 0.01]

TR Prec score: 1.0
TR F1 score: 1.0

Test f1: 0.844
with following performance in test:
             precision    recall  f1-score   support

          0       0.75      1.00      0.86         6
          1       1.00      0.71      0.83         7

avg / total       0.88      0.85      0.84        13

[[6 0]
 [2 5]]
Total time: 22.1901941299


In [48]:
print grid_pipeline.best_estimator_.get_params()

{'Imputer': TypeFeatImputer(allNameCols=array(['GENERO', 'ACTIVO', ..., 'EUROQOL_3', 'VISUAL EUROQOL_3'], dtype=object),
        dataCatCols=array([1, 1, ..., 0, 0])), 'nn__learning_rate_init': 0.001, 'Scaler__copy': True, 'nn__alpha': 0.01, 'Imputer__dataCatCols': array([1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0]), 'nn__shuffle': True, 'nn__nesterovs_momentum': True, 'nn__early_stopping': False, 'nn__hidden_layer_sizes': (70, 70), 'nn': MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(70, 70), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=13, shuffle=True,
       solver='lbfgs', tol=0.0001

In [49]:
y_pred = grid_pipeline.predict(X_test)
print y_pred
print ts_codes[y_pred == False]

[0 1 1 1 0 0 0 0 1 0 1 0 0]
['MY22' 'MY41' 'MY03' 'MY34' 'MY09' 'MY44' 'MY15' 'MY48']


In [50]:
# Compute pipeline evaluation with CV
print grid_pipeline.best_estimator_.get_params()
print "\nCV INNER selected params {}".format(grid_pipeline.best_params_)
print "CV INNER {} score: {}".format(metric, grid_pipeline.best_score_)

# Computel Train score (with best CV params)
y_pred = grid_pipeline.predict(X_train)                
train_scores = precision_score(y_train, y_pred, average='weighted', pos_label=None)
train_f1_scores = f1_score(y_train, y_pred, average='weighted', pos_label=None)

print "\nTR Prec score:", train_scores
print "TR F1 score:", train_f1_scores

#Compute test score
y_pred = grid_pipeline.predict(X_test)
test_f1 = f1_score(y_test, y_pred, average='weighted', pos_label=None)
print "\nTest f1: %0.3f" % (test_f1)
print "with following performance in test:"
print classification_report(y_test, y_pred)
print confusion_matrix(y_test, y_pred)

{'Imputer': TypeFeatImputer(allNameCols=array(['GENERO', 'ACTIVO', ..., 'EUROQOL_3', 'VISUAL EUROQOL_3'], dtype=object),
        dataCatCols=array([1, 1, ..., 0, 0])), 'nn__learning_rate_init': 0.001, 'Scaler__copy': True, 'nn__alpha': 0.01, 'Imputer__dataCatCols': array([1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0]), 'nn__shuffle': True, 'nn__nesterovs_momentum': True, 'nn__early_stopping': False, 'nn__hidden_layer_sizes': (70, 70), 'nn': MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(70, 70), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=13, shuffle=True,
       solver='lbfgs', tol=0.0001